In [3]:
import os
#import keras
import json
import subprocess

In [6]:
#整理文本为bert适合的格式
def process_text_for_bert(record_path = "//10.200.42.124/videos/segments",
                    ori_path = "data/text",
                    save_path = "data/bert/samples"):
    '''
    将精彩片段抽取出来，正例反例分开
    :param record_path:
    :param ori_path:
    :param save_path:
    :return:
    '''
    #with open("data/stopwords.txt",encoding='utf8') as f:
        #stop_words=set(f.read().split('\n'))
    stop_words = ['囡', '嗯', '吖', '唉', '呀']

    count=0
    if not os.path.exists(save_path):
        os.makedirs(save_path)
    pf=open(os.path.join(save_path,'positive.txt'),'w',encoding='utf8')
    nf=open(os.path.join(save_path,'negative.txt'),'w',encoding='utf8')
    for date_path in os.listdir(ori_path):
        for text_path in os.listdir(os.path.join(ori_path,date_path)):
            print("processing... ",ori_path,date_path,text_path)
            highlights = set()
            # if exists highlights
            if os.path.exists(os.path.join(record_path,date_path,text_path[:-4],'pos','p')):
                good_frames=os.listdir(os.path.join(record_path,date_path,text_path[:-4],'pos','p'))
                if not good_frames:
                    print("no highlights")
                    continue
                for frame in good_frames:
                    try:
                        minute=((int(frame.split('.')[0])-3)*10+9)//60
                    except Exception as e:
                        print(e)
                        print(frame)
                    else:
                        if minute<0:
                            minute=0
                        highlights.add(minute)
            # no highlights
            else:
                print("no highlights")
                continue
            print("highlights: ",highlights)
            count+=1
            with open(os.path.join(ori_path,date_path,text_path),encoding='utf8') as f:
                for i,line in enumerate(f):
                    line=line.strip()
                    if not line: # empty line (minute)
                        continue
                    #line=line.replace(' ','。')
                    for sw in stop_words:
                        line=line.replace(sw,'')
                    
                    if i in highlights:
                        pf.write(line+'\n')
                    else:
                        nf.write(line+'\n')
    pf.close()
    nf.close()
    print("number of valid files:",count)
    return 0

In [7]:
process_text_for_bert()

processing...  data/text records_20181009 10.152.29.61_3_10091730_10092030.txt
highlights:  {83, 84, 37, 38}
processing...  data/text records_20181009 10.152.29.61_3_10091730_10092030_1.txt
highlights:  {18}
processing...  data/text records_20181009 10.152.29.61_5_10091730_10092030.txt
highlights:  {83, 84, 37, 38}
processing...  data/text records_20181009 10.152.29.61_5_10091730_10092030_1.txt
highlights:  {18, 19}
processing...  data/text records_20181010 10.152.29.61_14_10101700_10101900.txt
highlights:  {12, 13, 14, 15, 16, 18, 19, 20, 21, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 41, 44, 45, 46, 51, 52, 53, 54, 81, 82, 84, 85, 86}
processing...  data/text records_20181010 10.152.29.61_21_10101700_10101900.txt
highlights:  {12, 13, 15, 16, 18, 19, 20, 21, 24, 25, 26, 28, 29, 30, 31, 32, 33, 34, 39, 40, 41, 43, 44, 46, 53, 54, 55, 81, 82, 83, 84, 85, 86}
processing...  data/text records_20181010 10.152.29.61_2_10101700_10101900.txt
highlights:  {34, 35, 37, 38, 40, 41, 10, 42, 43, 44,

Thumbs.db
highlights:  {16, 29, 30, 47}
processing...  data/text records_20181023 10.152.29.61_6_10231700_10231900.txt
highlights:  {35, 58, 99, 44, 109, 14, 111, 19, 20, 53, 119, 24, 25, 26, 59}
processing...  data/text records_20181024 10.152.29.61_14_10241700_10241900.txt
highlights:  {56, 58, 59, 42, 12, 13, 45, 18, 20, 21, 52, 23, 53, 54, 55, 27, 60, 61, 57}
processing...  data/text records_20181024 10.152.29.61_14_10241700_10241900_1.txt
no highlights
processing...  data/text records_20181024 10.152.29.61_15_10241645_10241815.txt
highlights:  {6}
processing...  data/text records_20181024 10.152.29.61_15_10241645_10241815_1.txt
no highlights
processing...  data/text records_20181024 10.152.29.61_6_10241700_10241900.txt
highlights:  {32, 33, 34, 36, 30, 31}
processing...  data/text records_20181024 Chinese.wav.ns_enh.analysiss.txt
no highlights
processing...  data/text records_20181025 10.152.29.61_3_10251730_10251930.txt
highlights:  {32, 33, 37, 74, 75, 44, 13, 45, 19, 20, 93, 22

0

In [8]:
#提取正样本特征
bert_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert"
bert_model_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert\\chinese_L-12_H-768_A-12"
output_path="data/bert/parameter/"
input_path="data/bert/samples/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
cmd="python %s/extract_features.py \
  --input_file=%s \
  --output_file=%s \
  --vocab_file=%s/vocab.txt \
  --bert_config_file=%s/bert_config.json \
  --init_checkpoint=%s/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=512 \
  --batch_size=8"%(bert_path, input_path+"positive.txt", output_path+"positive.json", bert_model_path, bert_model_path, bert_model_path)

subprocess.call(cmd)

0

In [9]:
#提取负样本特征
bert_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert"
bert_model_path="C:\\Users\\v_sunhao7\\PycharmProjects\\bert\\chinese_L-12_H-768_A-12"
output_path="data/bert/parameter/"
input_path="data/bert/samples/"
if not os.path.exists(output_path):
    os.makedirs(output_path)
cmd="python %s/extract_features.py \
  --input_file=%s \
  --output_file=%s \
  --vocab_file=%s/vocab.txt \
  --bert_config_file=%s/bert_config.json \
  --init_checkpoint=%s/bert_model.ckpt \
  --layers=-1 \
  --max_seq_length=512 \
  --batch_size=8"%(bert_path, input_path+"negative.txt", output_path+"negative.json", bert_model_path, bert_model_path, bert_model_path)

subprocess.call(cmd)

0

In [10]:
import numpy as np
import keras

In [12]:
with open("data/bert/parameter/positive.json", encoding='utf8') as f:
    for line in f:
        txt=line
        break

In [15]:
t=json.loads(txt)

In [19]:
t['features'][0].keys()

dict_keys(['token', 'layers'])

In [25]:
t['features'][0]['layers'][0]['values']

[0.26017,
 -0.639577,
 -1.191522,
 0.390413,
 0.347705,
 -0.817369,
 -0.74129,
 -0.208016,
 -1.048004,
 -0.997688,
 0.068266,
 -0.70002,
 -0.375047,
 -0.479586,
 1.73727,
 -0.133035,
 0.722432,
 -0.335764,
 -0.444146,
 -0.205821,
 -0.594827,
 0.021374,
 0.096013,
 0.665886,
 0.535247,
 0.103054,
 0.049016,
 -0.206277,
 1.473822,
 -0.63301,
 0.254615,
 -0.288945,
 0.271865,
 0.028365,
 0.478225,
 -0.980785,
 1.001239,
 -0.061165,
 0.466724,
 -0.205114,
 -0.168629,
 -0.659644,
 -0.286124,
 1.807682,
 0.630481,
 -0.703435,
 0.234511,
 0.163161,
 -0.380205,
 0.811586,
 0.070688,
 9.272715,
 0.635651,
 0.520954,
 -0.838365,
 0.603429,
 1.107339,
 -0.103124,
 0.350311,
 -0.540879,
 -0.572957,
 -1.523377,
 0.227953,
 1.020037,
 0.090945,
 -0.696282,
 0.306956,
 0.39359,
 -1.007883,
 0.174054,
 -0.007338,
 -0.46346,
 0.28427,
 0.317606,
 0.302067,
 1.427411,
 -0.79731,
 0.655787,
 0.718388,
 -0.679789,
 0.447854,
 0.480994,
 -0.324864,
 -0.338615,
 -0.226227,
 -0.11009,
 0.346057,
 -1.3926,
 -